# Amazon Scraper
#### A web scraper that helps me monitor the in stock status of animal crossing consoles.

In [312]:
# Python script for Amazon product availability checker 

# importing libraries 
from lxml import html 
import requests 
from time import sleep 
import time 
import schedule 
import datetime
import random

import re
import webbrowser
from bs4 import BeautifulSoup

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [283]:
# Email id for who want to check availability 
receiver_email_id = ""

# URL for different shops
amazonURL1 = "https://www.amazon.com/gp/product/B07X51X3CB/ref=crt_ewc_title_dp_2?ie=UTF8&psc=1&smid=ATVPDKIKX0DER" # For Testing
amazonURL = "https://www.amazon.com/Nintendo-Switch-Animal-Crossing-New-Horizons/dp/B084DDDNRP/index.html"

# Email
smtp_server = "smtp.gmail.com"
port = 587
password = ""
sender_email = ""
receiver_email = ""
receiver_email_list = ["",""]

In [284]:
def main():
    delay_time = random.randint(0,30)
    while(True):
        getAmazon()
        
        sleep(delay_time)
        
def getPage(url):
    print("Requesting page from: " + str(url))
    headers = {'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 12_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/80.0.3987.95 Mobile/15E148 Safari/605.1'}  
    page = requests.get(url, headers = headers)  
    return page

def outputHTML(html, site):
    with open(str(site+".html"), "w", encoding = 'utf-8') as file:
        file.write(str(html))

def mailman(store, url):
    # Message container.
    msg = MIMEMultipart('alternative')
    msg['Subject'] = "Item In Store!"
    msg['From'] = sender_email
    msg['To'] = receiver_email
    text = "In Stock at " + store + ".\n " + url
    msg.attach(MIMEText(text, 'plain'))
    print("Message created: \n" + text)

    # Login to SMTP and send email.
    s = smtplib.SMTP(smtp_server, port) 
    s.starttls() 
    s.login(sender_email,password)  
    for receiver in receiver_email_list:
        s.sendmail(sender_email, receiver, msg.as_string())
        print("Sent email to: " + receiver)
    s.quit()

In [285]:
def getAmazon():
    # Parse HTML
    print("-----------NEW SESSION: " + str(datetime.datetime.now()) + "-----------")
    page = getPage(amazonURL)
    soup = BeautifulSoup(page.content, 'lxml')
    
    # Output received HTML for debug
    outputHTML(soup, "Amazon")
    
    # Determine whether is in stock
    if(soup.find('a', class_='a-button-text') == None):
        print("Amazon In Stock!")
        
        print("Opening up page on local computer!")
        webbrowser.open(amazonURL)
        
        print("Sending Email to everyone in mailing list!")
        mailman("Amazon",amazonURL)
    elif("See All Buying Options" in soup.find('a', class_='a-button-text').text):
        print("Amazon Out of Stock!")
    else:
        print("UnknownErrorException")
        
    print("------------------------END OF SESSION------------------------")